# COMP3221 - Week 9 - Security

In [ ]:
!git clone https://github.com/longtanle/distributed_systems_tutorials
%cd distributed_systems_tutorials/w9

The goal of this tutorial is to better understand the main technique to achieve security and to use cryptographic signatures to verify the ownership of blockchain assets.

## **Digital Signatures**

### Overview

<img src="https://alexromanov.github.io/img/20220515/howsignworks.png" width=600>

A special mathematical algorithm that allows checking the **integrity** and **authenticity** of messages.


1. **Integrity**: The recipient and sender ensure that the message has not been modified during transmission → using Hash Function


1. **Authenticity**: The recipient be sure that the sender sent the message → using Asymmetric Encryption




<img src="https://pub.mdpi-res.com/cryptography/cryptography-06-00025/article_deploy/html/images/cryptography-06-00025-g001-550.jpg?1652940133" width=500>

There are many algorithms for digital signatures: **RSA-PSS**, **DSA**, **ECDSA**, **Rabin signature**, and **Schnorr signature**.

#### **Question**: How to distribute Public Key?

<img src="https://blog.cloudflare.com/content/images/2017/09/image10.png" width=500>

### Digital Signature - Usage in Blockchain

<img src="https://alexromanov.github.io/img/20220515/signblockchain.png" width=700>

Every time you make a new transaction in the wallet, the transaction is signed with your key.

- In the blockchain, you can check who created one or the other transaction.

- Everybody in the blockchain can check and prove that particular transaction was sent by a user with the correct private key.

Additionally, each new block is signed by a miner of this block.

**Bitcoin** initially used the **ECDSA** algorithm for signing transactions, and now using the **Schnorr signature**.

**Ethereum** currently uses **ECDSA**.

**Cardano** uses **Ed25519**.

## **Exercise 1: Signature**

Use the **ed25519** digital signature algorithm to sign a request and verify that the signature is correct.

**ed25519** is a digital signature scheme based on elliptic curves.
We will use the cryptography Python library to create and validate **ed25519**  signatures.

<img src="https://blog.mozilla.org/warner/files/2011/11/key-formats.png" width=500>

Install the library in your Python environment.

In [ ]:
!pip install cryptography


Execute the following example. We import the private key class, generate a random private
key, and generate a signature for an example message. Then, we derive the public key for
the private key that we used to sign the message, and verify the signature with the public
key.

In [ ]:
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

private_key = Ed25519PrivateKey.generate()
signature = private_key.sign(b"my authenticated message")
public_key = private_key.public_key()

print(private_key.private_bytes_raw().hex())
print(public_key.public_bytes_raw().hex())

5457c383eee393e399b2effa1033b37c7710875ab88f704634dc3fed371240ed
390e7701bf85a999cab65d3691f7af0ba7140c630ded545fbcaf39c0df68e826


In [ ]:
# Raises InvalidSignature if verification fails
print(public_key.verify(signature, b"my authenticated message"))

None


In [ ]:
# Raises InvalidSignature if verification fails
print(public_key.verify(signature, b"not my authenticated message"))

InvalidSignature: 

## **Exercise 2: Authentication in Blockchain**

Using the signature above, make sure that your blockchain updates a data item only if the update is correctly signed by the owner of this data item.

In Lab 7, we defined transaction as follows.

In [ ]:
transaction = {
    'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
    'message': 'hello'
}

### Task 2.1 - Signing

Now, we want to sign the transaction and ensure the message authenticity in the blockchain. As with the hashes, we need to *define a deterministic string representation for the transaction that will be signed*. An example of such representation is similar to the one we used for the block.

First, we define a Python dictionary for the transaction, and use the JSON library to create an object with lexicographically sorted dictionary keys.

Integrate the digital signatures into your blockchain implementation from Lab 7. Define a signature based on the sender and message transaction field keys, and include a signature into the message. On the blockchain side, modify the verification function to decode the sender as a public key, and verify the included signature against the message and the sender public key. Consider the signature verification in conjunction with other verification criteria.

In [ ]:
from cryptography.exceptions import InvalidSignature
import cryptography.hazmat.primitives.asymmetric.ed25519 as ed25519
from enum import Enum
import hashlib
import json
import re

sender_valid = re.compile('^[a-fA-F0-9]{64}$')
# signature_valid = re.compile()

TransactionValidationError = Enum('TransactionValidationError', ['INVALID_JSON', 'INVALID_SENDER', 'INVALID_MESSAGE', 'INVALID_SIGNATURE'])

def make_transaction(sender, message, signature) -> str:
	return json.dumps({'sender': sender, 'message': message, 'signature': signature})

def transaction_bytes(transaction: dict) -> bytes:
	# TODO return JSON string of the transaction as bytes, using json.dumps with sort_keys=True, including only 'sender' and 'message'
	pass

def make_signature(private_key: ed25519.Ed25519PrivateKey, message: str) -> str:
	# TODO create a transaction dictionary with the derived public key from private key as 'sender', and message
	# return hex representation of signature, created by calling private_key.sign and using transaction_bytes function
	pass

def validate_transaction(transaction: str) -> dict | TransactionValidationError:
	try:
		tx = json.loads(transaction)
	except json.JSONDecodeError:
		return TransactionValidationError.INVALID_JSON

	if not(tx.get('sender') and isinstance(tx['sender'], str) and sender_valid.search(tx['sender'])):
		return TransactionValidationError.INVALID_SENDER

	if not(tx.get('message') and isinstance(tx['message'], str) and len(tx['message']) <= 70 and tx['message'].isalnum()):
		return TransactionValidationError.INVALID_MESSAGE

	# TODO verify signature and return INVALID_SIGNATURE if signature verification fails
	# use ed25519.Ed25519PublicKey.from_public_bytes to get Ed25519PublicKey object from 'sender' attribute
	# verify 'signature' field similar to 'sender', requiring a hex string of length 128
	# call public_key.verify to verify the signature. catch InvalidSignature if it is raised

	return tx

In [ ]:
class Blockchain():
	def  __init__(self):
		self.blockchain = []
		self.pool = []
		self.new_block('0' * 64)

	def new_block(self, previous_hash=None):
		block = {
			'index': len(self.blockchain) + 1,
			'transactions': self.pool.copy(),
			'previous_hash': previous_hash or self.blockchain[-1]['current_hash'],
		}
		block['current_hash'] = self.calculate_hash(block)
		self.pool = []
		self.blockchain.append(block)

	def last_block(self):
		return self.blockchain[-1]

	def calculate_hash(self, block: dict) -> str:
		block_object: str = json.dumps({k: block.get(k) for k in ['index', 'transactions', 'previous_hash']}, sort_keys=True)
		block_string = block_object.encode()
		raw_hash = hashlib.sha256(block_string)
		hex_hash = raw_hash.hexdigest()
		return hex_hash

	def add_transaction(self, transaction: str) -> bool:
		if isinstance((tx := validate_transaction(transaction)), dict):
			self.pool.append(tx)
			return True
		return False

#### Solution

In [ ]:
from cryptography.exceptions import InvalidSignature
import cryptography.hazmat.primitives.asymmetric.ed25519 as ed25519
from enum import Enum
import hashlib
import json
import re

sender_valid = re.compile('^[a-fA-F0-9]{64}$')
signature_valid = re.compile('^[a-fA-F0-9]{128}$')

TransactionValidationError = Enum('TransactionValidationError', ['INVALID_JSON', 'INVALID_SENDER', 'INVALID_MESSAGE', 'INVALID_SIGNATURE'])

def make_transaction(sender, message, signature) -> str:
	return json.dumps({'sender': sender, 'message': message, 'signature': signature})

def transaction_bytes(transaction: dict) -> bytes:
	return json.dumps({k: transaction.get(k) for k in ['sender', 'message']}, sort_keys=True).encode()

def make_signature(private_key: ed25519.Ed25519PrivateKey, message: str) -> str:
	transaction = {'sender': private_key.public_key().public_bytes_raw().hex(), 'message': message}
	return private_key.sign(transaction_bytes(transaction)).hex()

def validate_transaction(transaction: str) -> dict | TransactionValidationError:
	try:
		tx = json.loads(transaction)
	except json.JSONDecodeError:
		return TransactionValidationError.INVALID_JSON

	if not(tx.get('sender') and isinstance(tx['sender'], str) and sender_valid.search(tx['sender'])):
		return TransactionValidationError.INVALID_SENDER

	if not(tx.get('message') and isinstance(tx['message'], str) and len(tx['message']) <= 70 and tx['message'].isalnum()):
		return TransactionValidationError.INVALID_MESSAGE

	public_key = ed25519.Ed25519PublicKey.from_public_bytes(bytes.fromhex(tx['sender']))
	if not(tx.get('signature') and isinstance(tx['signature'], str) and signature_valid.search(tx['signature'])):
		return TransactionValidationError.INVALID_SIGNATURE
	try:
		public_key.verify(bytes.fromhex(tx['signature']), transaction_bytes(tx))
	except InvalidSignature:
		return TransactionValidationError.INVALID_SIGNATURE

	return tx


class Blockchain():
	def  __init__(self):
		self.blockchain = []
		self.pool = []
		self.new_block('0' * 64)

	def new_block(self, previous_hash=None):
		block = {
			'index': len(self.blockchain) + 1,
			'transactions': self.pool.copy(),
			'previous_hash': previous_hash or self.blockchain[-1]['current_hash'],
		}
		block['current_hash'] = self.calculate_hash(block)
		self.pool = []
		self.blockchain.append(block)

	def last_block(self):
		return self.blockchain[-1]

	def calculate_hash(self, block: dict) -> str:
		block_object: str = json.dumps({k: block.get(k) for k in ['index', 'transactions', 'previous_hash']}, sort_keys=True)
		block_string = block_object.encode()
		raw_hash = hashlib.sha256(block_string)
		hex_hash = raw_hash.hexdigest()
		return hex_hash

	def add_transaction(self, transaction: str) -> bool:
		if isinstance((tx := validate_transaction(transaction)), dict):
			self.pool.append(tx)
			return True
		return False


### Task 2.2 – Testing

Here is a private key:
`6dee02b55d8914c145568cb3f3b84586ead2a85910f5b062d7f3f29ddcb4c7aa`

Use this private key in the following example to test if you implemented the signature generation correctly.

```
{
    'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
    'message': 'hello',
    'signature': '402f2e4de75883ae7350335206d0e903072b0790226cf9a536b475e028557265eb0712827a273ef422ca7247c5fbd86919c0bfa6c6067364de6b1366a1702b0b'
}
```

Create private key object from the hex string

In [ ]:
privated_hex = "6dee02b55d8914c145568cb3f3b84586ead2a85910f5b062d7f3f29ddcb4c7aa"
loaded_private_key = Ed25519PrivateKey.from_private_bytes(bytes.fromhex(privated_hex))
loaded_private_key

Get the public key string from the private key

In [ ]:
loaded_private_key.public_key().public_bytes_raw().hex()

'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b'

Generate signature using `make_signature` function

In [ ]:
message = 'hello'
signature = make_signature(loaded_private_key, message)
signature

'ea2d7f7f72849fae4c1d5bf3ea74b10dc64a645790f05e231cb4bddc67d7efbc0e2eca70392b445f834cab13053687faaf7ff1b385c2890628284e2161bfcb0d'

Verify the sample transaction

In [ ]:
sender = 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b'
message = 'hello'
signature = '402f2e4de75883ae7350335206d0e903072b0790226cf9a536b475e028557265eb0712827a273ef422ca7247c5fbd86919c0bfa6c6067364de6b1366a1702b0b'

transaction = make_transaction(sender, message, signature)
validate_transaction(transaction)

<TransactionValidationError.INVALID_SIGNATURE: 4>

Verify the new transaction

In [ ]:
sender = 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b'
message = 'hello'
signature = 'ea2d7f7f72849fae4c1d5bf3ea74b10dc64a645790f05e231cb4bddc67d7efbc0e2eca70392b445f834cab13053687faaf7ff1b385c2890628284e2161bfcb0d'

transaction = make_transaction(sender, message, signature)
validate_transaction(transaction)

{'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
 'message': 'hello',
 'signature': 'ea2d7f7f72849fae4c1d5bf3ea74b10dc64a645790f05e231cb4bddc67d7efbc0e2eca70392b445f834cab13053687faaf7ff1b385c2890628284e2161bfcb0d'}

### Test Blockchain

In [ ]:
# Create a block as dictionary
B = Blockchain()
print(B.blockchain)

[{'index': 1, 'transactions': [], 'previous_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'current_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1'}]


Create a new private-public key pair for the transaction

In [ ]:
private_key = Ed25519PrivateKey.generate()
public_key = private_key.public_key()

public_hex = public_key.public_bytes_raw().hex()
public_hex

'f314b3931b9e69721c7458e8da1fbfa4606943e27b0ff53a4c90cfd274f82a3c'

Generate signature for the message

In [ ]:
sender = public_hex
message = 'Transaction'

signature = make_signature(private_key, message)
signature

'887f57bb11cebee8746dab11d87e1c6536e7ccdc2e66da93b38825dcc8f48d11debe3dff966b3c30004a6d7b4d747f462bfd8583a5fba546f0d34a4ac10c8a04'

Create transaction

In [ ]:
transactions = make_transaction(sender, message, signature)
transactions

'{"sender": "f314b3931b9e69721c7458e8da1fbfa4606943e27b0ff53a4c90cfd274f82a3c", "message": "Transaction", "signature": "887f57bb11cebee8746dab11d87e1c6536e7ccdc2e66da93b38825dcc8f48d11debe3dff966b3c30004a6d7b4d747f462bfd8583a5fba546f0d34a4ac10c8a04"}'

Add transaction to the pool

In [ ]:
B.add_transaction(transactions)
print(B.pool)

[{'sender': 'f314b3931b9e69721c7458e8da1fbfa4606943e27b0ff53a4c90cfd274f82a3c', 'message': 'Transaction', 'signature': '887f57bb11cebee8746dab11d87e1c6536e7ccdc2e66da93b38825dcc8f48d11debe3dff966b3c30004a6d7b4d747f462bfd8583a5fba546f0d34a4ac10c8a04'}]


Add new block

In [ ]:
print("Add new block")
B.new_block()
print(B.blockchain)

Add new block
[{'index': 1, 'transactions': [], 'previous_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'current_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1'}, {'index': 2, 'transactions': [{'sender': 'f314b3931b9e69721c7458e8da1fbfa4606943e27b0ff53a4c90cfd274f82a3c', 'message': 'Transaction', 'signature': '887f57bb11cebee8746dab11d87e1c6536e7ccdc2e66da93b38825dcc8f48d11debe3dff966b3c30004a6d7b4d747f462bfd8583a5fba546f0d34a4ac10c8a04'}], 'previous_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1', 'current_hash': '367efaf3b692df8d01095ec0ffdb74d5afff6f406b65d6082e500499eae437af'}]
